In [ ]:
"""
VIDEO to TEXT STEGANOGRAPHY (Glyph Perturbation Cardinality, GPC)

This script embeds spatio-temporal video data into rendered text glyphs
using deterministic raster-domain perturbation cardinality. Each video
frame is processed independently, without temporal coupling.

Each frame is resized to a canonical resolution and treated as an RGB
image. Pixel intensities are quantized to integer payloads v ∈ [0, 26],
embedded into glyph interiors via pixel perturbation, and reconstructed
by counting glyph-level differences.

This implementation additionally reports cover-text utilization,
explicitly tracking how many cover glyphs are available, used, and unused.
"""

# ============================================================
# IMPORTS
# ============================================================

import os
import random
import numpy as np
import cv2
from PIL import Image, ImageDraw, ImageFont
from skimage.metrics import structural_similarity as ssim

# ============================================================
# CONFIGURATION
# ============================================================

VIDEO_PATH = "Video_1.mp4"   # Input Video
OUT_DIR = "video_to_text_full"   # Output Directory
os.makedirs(OUT_DIR, exist_ok=True)

FRAME_SIZE = (120, 120)   # canonical paper resolution
TILE_W, TILE_H = 40, 50
FONT_SIZE = 36

P_MAX = 26
DELTA = 1
SEED = 42

BG = 255
INK = 0

COVER_TEXT = (
    "Beneath the midnight sea, a pearl city glowed "
    "and memories drifted upward into silence "
) * 50000

# ============================================================
# FONT
# ============================================================

def load_font(size):
    for p in [
        "arial.ttf",
        "DejaVuSansMono.ttf",
        "/usr/share/fonts/truetype/dejavu/DejaVuSansMono.ttf"
    ]:
        try:
            return ImageFont.truetype(p, size)
        except Exception:
            continue
    return ImageFont.load_default()

FONT = load_font(FONT_SIZE)

# ============================================================
# COVER TEXT UTILITIES
# ============================================================

def extract_cover_letters(text: str):
    return [c for c in text.upper() if c.isalpha()]

ALL_COVER_LETTERS = extract_cover_letters(COVER_TEXT)
TOTAL_COVER_GLYPHS = len(ALL_COVER_LETTERS)

if TOTAL_COVER_GLYPHS == 0:
    raise ValueError("COVER_TEXT contains no alphabetic glyphs.")

# ============================================================
# RASTER UTILITIES
# ============================================================

def arrange_tiles(tiles, per_row=80):
    blank = np.full((TILE_H, TILE_W), BG, np.uint8)
    rows = []
    for i in range(0, len(tiles), per_row):
        row = tiles[i:i + per_row]
        if len(row) < per_row:
            row += [blank] * (per_row - len(row))
        rows.append(np.hstack(row))
    return np.vstack(rows)

def rasterize_letter(ch):
    img = Image.new("L", (TILE_W, TILE_H), BG)
    d = ImageDraw.Draw(img)
    bbox = d.textbbox((0, 0), ch, font=FONT)
    w, h = bbox[2] - bbox[0], bbox[3] - bbox[1]
    d.text(((TILE_W - w)//2, (TILE_H - h)//2), ch, fill=INK, font=FONT)
    return np.asarray(img, np.uint8)

def encode_glyph(can, v, rng):
    enc = can.copy()
    pts = list(zip(*np.where(can == INK)))
    for r, c in rng.sample(pts, min(v, len(pts))):
        enc[r, c] = INK + DELTA
    return enc

def decode_glyph(can, enc):
    return int(((can == INK) & (enc == INK + DELTA)).sum())

# ============================================================
# METRICS
# ============================================================

def mse(a, b): return np.mean((a.astype(float) - b.astype(float))**2)
def mae(a, b): return np.mean(np.abs(a.astype(float) - b.astype(float)))
def rmse(a, b): return np.sqrt(mse(a, b))
def psnr(a, b):
    m = mse(a, b)
    return np.inf if m == 0 else 20 * np.log10(255 / np.sqrt(m))

def ssim_rgb(a, b):
    sr = ssim(a[:, :, 0], b[:, :, 0], data_range=255)
    sg = ssim(a[:, :, 1], b[:, :, 1], data_range=255)
    sb = ssim(a[:, :, 2], b[:, :, 2], data_range=255)
    return (sr + sg + sb) / 3, (sr, sg, sb)

# ============================================================
# FRAME PIPELINE
# ============================================================

def encode_frame(frame_rgb):
    rng = random.Random(SEED)
    norm = lambda ch: np.rint(ch / 255 * P_MAX).astype(int).flatten()

    payloads = [norm(frame_rgb[:, :, i]) for i in range(3)]
    glyphs_per_channel = len(payloads[0])

    letters = ALL_COVER_LETTERS.copy()
    while len(letters) < glyphs_per_channel:
        letters += ALL_COVER_LETTERS
    letters = letters[:glyphs_per_channel]

    canonicals, encoded = [], []
    for vals in payloads:
        can_tiles, enc_tiles = [], []
        for ch, v in zip(letters, vals):
            can = rasterize_letter(ch)
            enc = encode_glyph(can, v, rng)
            can_tiles.append(can)
            enc_tiles.append(enc)
        canonicals.append(can_tiles)
        encoded.append(enc_tiles)

    return canonicals, encoded, glyphs_per_channel

def decode_frame(canonicals, encoded):
    chans = []
    for can_tiles, enc_tiles in zip(canonicals, encoded):
        vals = np.array([decode_glyph(c, e) for c, e in zip(can_tiles, enc_tiles)])
        chans.append((vals.reshape(FRAME_SIZE) / P_MAX * 255).astype(np.uint8))
    return np.stack(chans, axis=-1)

# ============================================================
# MAIN VIDEO LOOP
# ============================================================

cap = cv2.VideoCapture(VIDEO_PATH)
fps = cap.get(cv2.CAP_PROP_FPS)
if fps <= 0 or np.isnan(fps):
    fps = 30.0

video_writer = cv2.VideoWriter(
    os.path.join(OUT_DIR, "decoded_video.mp4"),
    cv2.VideoWriter_fourcc(*"mp4v"),
    fps,
    FRAME_SIZE
)

frame_idx = 0
metrics_all = []
total_glyph_embeddings = 0

os.makedirs(f"{OUT_DIR}/frames", exist_ok=True)
os.makedirs(f"{OUT_DIR}/glyphs", exist_ok=True)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.resize(frame, FRAME_SIZE)
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    canonicals, encoded, glyphs_used = encode_frame(frame_rgb)
    decoded = decode_frame(canonicals, encoded)

    total_glyph_embeddings += glyphs_used * 3  # R,G,B

    Image.fromarray(frame_rgb).save(f"{OUT_DIR}/frames/frame_{frame_idx:04d}_canonical.png")
    Image.fromarray(decoded).save(f"{OUT_DIR}/frames/frame_{frame_idx:04d}_decoded.png")

    video_writer.write(cv2.cvtColor(decoded, cv2.COLOR_RGB2BGR))

    for name, can_tiles, enc_tiles in zip(["R", "G", "B"], canonicals, encoded):
        Image.fromarray(arrange_tiles(can_tiles)).save(
            f"{OUT_DIR}/glyphs/frame_{frame_idx:04d}_{name}_canonical.png"
        )
        Image.fromarray(arrange_tiles(enc_tiles)).save(
            f"{OUT_DIR}/glyphs/frame_{frame_idx:04d}_{name}_encoded.png"
        )
        diff_tiles = [((c == INK) & (e == INK + DELTA)).astype(np.uint8) * 255
                      for c, e in zip(can_tiles, enc_tiles)]
        Image.fromarray(arrange_tiles(diff_tiles)).save(
            f"{OUT_DIR}/glyphs/frame_{frame_idx:04d}_{name}_difference.png"
        )

    M = mse(frame_rgb, decoded)
    A = mae(frame_rgb, decoded)
    R = rmse(frame_rgb, decoded)
    P = psnr(frame_rgb, decoded)
    SS, _ = ssim_rgb(frame_rgb, decoded)

    metrics_all.append((M, A, R, P, SS))
    frame_idx += 1

cap.release()
video_writer.release()

# ============================================================
# METRICS + COVER REPORT
# ============================================================

metrics_all = np.array(metrics_all)

reuse_factor = total_glyph_embeddings / TOTAL_COVER_GLYPHS

with open(f"{OUT_DIR}/metrics.txt", "w", encoding="utf-8") as f:
    f.write("VIDEO TO TEXT (GPC) METRICS\n")
    f.write(f"Frames processed: {len(metrics_all)}\n\n")
    f.write(f"MSE  (mean): {metrics_all[:,0].mean()}\n")
    f.write(f"MAE  (mean): {metrics_all[:,1].mean()}\n")
    f.write(f"RMSE (mean): {metrics_all[:,2].mean()}\n")
    f.write(f"PSNR (mean): {metrics_all[:,3].mean()}\n")
    f.write(f"SSIM (mean): {metrics_all[:,4].mean()}\n")

with open(f"{OUT_DIR}/cover_usage.txt", "w", encoding="utf-8") as f:
    f.write("VIDEO TO TEXT (GPC) - COVER TEXT USAGE\n")
    f.write(f"Unique cover glyphs available : {TOTAL_COVER_GLYPHS}\n")
    f.write(f"Total glyph embeddings        : {total_glyph_embeddings}\n")
    f.write(f"Reuse factor                  : {reuse_factor:.2f}x\n")

print("\n✓ Full video processed successfully")
print("✓ Decoded video saved as decoded_video.mp4")
print("✓ Frames, glyph rasters, difference maps saved")
print("✓ Metrics and cover usage report generated")
